In [1]:
import time
import socket
from queue import Queue 
from threading import Thread, Condition
from ipywidgets import widgets
from IPython.display import display
import random

In [2]:
class EXP_Output:
        def __init__(self):
            #self.out = widgets.Output(layout={'border': '1px solid black'})
            #display(self.out)
            #with self.out:
            #print(\"Experiment outputs\")
    
            self.Queue_Print = Queue()
            self.thread = Thread(target=self.loop_Print)
            self.thread.start()
    
        def print(self, msg):
            text_out = time.strftime("%H:%M:%S", time.localtime()) + "> "
            self.Queue_Print.put(text_out+msg)
    
        def loop_Print(self):
            while True:
                msg = self.Queue_Print.get()
                #with self.out:
                print(msg)
                self.Queue_Print.task_done()

In [3]:
class ALE_TextInput:
    
        def __init__(self, exp_out):
            
            self.Queue_User = Queue()
            self.exp_out = exp_out
    
            # Create a text input widget
            self.Text = widgets.Text(
                            value='',
                            placeholder='Type something and press enter',
                            description='To send:',
                            disabled=False
                        )
    
            # create a button object
            self.Botton =  widgets.Button(
                                description='Submit',
                                disabled=False,
                                button_style='', 
                                tooltip='Click me to submit a string',
                            )
    
            # Define a function to handle the click action on the button
            def on_submit_button_clicked(b):
    
                global thread_running
                
                msg = self.Text.value
                #self.exp_out.print(\"ALE_0: received input: \"+str(msg))
                    
                if thread_running == True:
                    if msg == "end":
                        thread_running = False
                    
                    for i in range(20):
                        self.Queue_User.put(msg+str(i))
    
                self.Text.value = ''  # Clear the input field after submission
    
            # Attach the event to the button
            self.Botton.on_click(on_submit_button_clicked)
            
            # Display the widgets
            display(self.Text, self.Botton)
            
        # to get a message from user queue. this function can block the thread
        def get(self):
            return self.Queue_User.get()

In [4]:
class ALE_TR:
    
    def __init__(self, name, upper_Tx, lower_TR, exp_out):
        
        self.Name     = name
        self.Upper_Tx = upper_Tx
        self.Lower_TR = lower_TR
        self.exp_out  = exp_out
        
    def loop_Tx(self):
        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Tx starting")
    
        while (thread_running == True):
            c = c + 1
            msg = self.Upper_Tx.get()
            text_out = self.Name + " Tx: message " + str(c) + ": " + str(msg)
            self.exp_out.print(text_out)
            self.Lower_TR.send(msg)
            
    def loop_Rx(self):
        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Rx starting")
    
        while (thread_running == True):
            c = c + 1
            msg = self.Lower_TR.receive()
            text_out = self.Name + " Rx: message " + str(c) + ": " + str(msg)
            self.exp_out.print(text_out)

In [5]:
STATE_READY_TO_SEND = 0
STATE_WAITING_ACK   = 1

EVENT_UPPER_TX  = 0
EVENT_LOWER_DAT = 1
EVENT_LOWER_ACK = 2
EVENT_TIMEOUT   = 3
        
class DLE_TR_FSM:
    
    def __init__(self, name, lower_TR, exp_out):
        
        self.Name      = name
        self.Lower_TR  = lower_TR
        self.Queue_Tx  = Queue()
        self.Queue_Rx  = Queue()
        self.exp_out   = exp_out
        
        self.Queue_FSM = Queue()
        
        self.cv_TxOp = Condition()
        self.TxOp    = True

        self.State = STATE_READY_TO_SEND
        
        self.Procedure = [[self.FSM_upper_Tx, self.FSM_abnormal],
                          [self.FSM_lower_Rx, self.FSM_lower_Rx],
                          [self.FSM_abnormal, self.FSM_lower_Rx_ack],
                          [self.FSM_abnormal, self.FSM_timeout]] 
        

        self.cv_Timer      = Condition()
        self.timer_active  = False
        self.timer_counter = 0
        
        self.tx_buffer  = ''
        self.tx_seq_num = '0'
        
        self.tx_ack_num = '0'
        
        
    def loop_Tx(self):
        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Tx starting")
    
        while (thread_running == True):
            with self.cv_TxOp: 
                while (self.TxOp != True): 
                    self.cv_TxOp.wait()
            
                self.TxOp = False
                c = c + 1
                msg = self.Queue_Tx.get()
                text_out = self.Name + " Tx: message " + str(c) + ": " + str(msg)
                self.exp_out.print(text_out)
                self.event_add(EVENT_UPPER_TX, msg) 

    def loop_Rx(self):
        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Rx starting")
    
        while (thread_running == True):
            c = c + 1
            msg = self.Lower_TR.receive()
            text_out = self.Name + " Rx: message " + str(c) + ": " + str(msg)
            self.exp_out.print(text_out)
            
            msg_type = msg[0]
            if (msg_type == '0'):
                self.event_add(EVENT_LOWER_DAT, msg[1:])
            elif (msg_type == '1'):
                self.event_add(EVENT_LOWER_ACK, msg[1:])
            else:
                text_out = self.Name + " Rx: message type unknown " + msg_type
                self.exp_out.print(text_out)

    def receive(self):
        return self.Queue_Rx.get()
    
    def send(self, msg):
        self.Queue_Tx.put(msg)
        
    def event_add(self, ev_type, msg):
        if (isinstance(msg, str)):
            msg = msg.encode()
        
        event = ev_type.to_bytes(1, "big")+msg
        self.Queue_FSM.put(event)        
        

    def loop_FSM(self):
        global thread_running
        self.exp_out.print(self.Name + ": loop_FSM starting")
        
        while (thread_running == True):
            event = self.Queue_FSM.get()
            ev_type = event[0]
            text_out = self.Name + " FSM: state: " + str(self.State)+" event type: " + str(ev_type)
            self.exp_out.print(text_out)
            msg = event[1:].decode('utf-8')
            self.Procedure[ev_type][self.State](msg)
        
    def FSM_abnormal(self, msg):
        text_out = self.Name + " FSM: error! " + msg
        self.exp_out.print(text_out)
    
    def FSM_upper_Tx(self, msg):   
        text_out = self.Name + " FSM: to send: " + msg
        self.exp_out.print(text_out)

        self.tx_buffer = msg
        
        msg = '0' + self.tx_seq_num + msg
        self.Lower_TR.send(msg)

        self.State = STATE_WAITING_ACK
        
        with self.cv_Timer:
            self.timer_active  = True
            self.timer_counter = 10
            self.cv_Timer.notify()    
    
    def FSM_lower_Rx(self, msg):
        seq = msg[0]
        msg = msg[1:]
        
        text_out = self.Name + " FSM: received: " + msg
        self.exp_out.print(text_out)

        if (seq == self.tx_ack_num):
            self.Queue_Rx.put(msg)
            self.tx_ack_num = chr(ord(self.tx_ack_num) + 1)
            if (self.tx_ack_num == chr(ord('9') + 1)):
                self.tx_ack_num = '0'
        
        else:
            text_out  = self.Name + " FSM: received received frame " + seq
            text_out += " but expected " + self.tx_ack_num
            self.exp_out.print(text_out)
            
        self.Lower_TR.send('1'+seq)

                
    def FSM_lower_Rx_ack(self, msg):
        ack = msg[0]
        if (ack != self.tx_seq_num):
            text_out = "ACK # " + ack + " does not match local seq # " + self.tx_seq_num
            self.exp_out.print(text_out)
            return

        self.timer_active = False
        
        self.tx_seq_num = chr(ord(self.tx_seq_num) + 1)
        if (self.tx_seq_num == chr(ord('9') + 1)):
                self.tx_seq_num = '0'
        
        with self.cv_TxOp:
            self.State = STATE_READY_TO_SEND
            self.TxOp  = True
            self.cv_TxOp.notify()
            
    def FSM_timeout(self, msg):
        text_out = self.Name + " FSM: to resend frame " + self.tx_seq_num + " " + self.tx_buffer
        self.exp_out.print(text_out)
        
        msg = '0' + self.tx_seq_num + self.tx_buffer
        self.Lower_TR.send(msg)
        
        with self.cv_Timer:
            self.timer_active  = True
            self.timer_counter = 10
            self.cv_Timer.notify()

    def loop_timer(self):
        global thread_running
        self.exp_out.print(self.Name + ": loop_Timer starting")
        
        while (thread_running == True):
            with self.cv_Timer: 
                while (self.timer_active == False): 
                    self.cv_Timer.wait() 
            
            time.sleep(0.5)

            if (self.timer_counter == 0):
                self.event_add(EVENT_TIMEOUT, 'x')
                self.timer_active = False

            else:
                self.timer_counter = self.timer_counter - 1

In [6]:
class PLE_TR:

    def __init__(self, name, Socket, AP_Tx, AP_Rx, exp_out):
        
        self.Name      = name
        self.Socket    = Socket
        self.AP_Tx     = AP_Tx
        self.AP_Rx     = AP_Rx
        self.Queue_Tx  = Queue()
        self.Queue_Rx  = Queue()
        self.exp_out   = exp_out
        
    def loop_Tx(self):
        global thread_running
        self.exp_out.print(self.Name + ": loop_Tx starting")
    
        while (thread_running == True):
            msg = self.Queue_Tx.get()
            text_out = self.Name + " Tx: message: " + str(msg)
            self.exp_out.print(text_out)
            msg_bytes = str.encode(msg)
            
            rand_val = random.random()
            
            if rand_val < 0.15:  # 15% packet loss
                self.exp_out.print("packet loss")
            elif rand_val < 0.20:  # 5% packet duplication
                self.exp_out.print("packet duplication")
                # Send original packet
                self.Socket.sendto(msg_bytes, self.AP_Tx)
                # Send duplicate packet
                self.Socket.sendto(msg_bytes, self.AP_Tx)
            else:  # 80% normal transmission
                self.Socket.sendto(msg_bytes, self.AP_Tx)

    def loop_Rx(self):
        global thread_running
        global bufferSize
        self.exp_out.print(self.Name + ": loop_Rx starting")
    
        self.Socket.bind(self.AP_Rx)
        
        while (thread_running == True):
            msg_addr = self.Socket.recvfrom(bufferSize)

            # Random delay between 0 and 3 seconds
            delay_time = random.uniform(0, 3)
            time.sleep(delay_time)
            self.exp_out.print(f"delayed for {delay_time:.2f} seconds")
        
            msg  = msg_addr[0].decode('utf-8')
            addr = msg_addr[1]
            text_out = self.Name + " Rx: from " + str(addr) + ": " + str(msg)
            self.exp_out.print(text_out)
            self.Queue_Rx.put(msg)
            
    def send(self, msg):
        self.Queue_Tx.put(msg)
        
    def receive(self):
        return self.Queue_Rx.get()

In [ ]:
class MLE_TR:
    
    def __init__(self, name, lower_TR, exp_out, experiment_id, is_sender=False):
        
        self.Name = name
        self.Lower_TR = lower_TR
        self.exp_out = exp_out
        self.experiment_id = experiment_id
        self.is_sender = is_sender
        
        # Statistics tracking
        self.sent_messages = {}  # msg_id -> timestamp
        self.received_messages = {}  # msg_id -> [timestamps]
        self.total_sent = 0
        self.total_received = 0
        self.duplicate_count = 0
        
        # Only create button for sender nodes
        if self.is_sender:
            # Create button widget for sending test messages
            self.test_button = widgets.Button(
                description='Start Test',
                disabled=False
            )
            
            # Define callback function for button click
            def on_test_button_clicked(b):
                self.send_test_messages()
            
            # Attach the event to the button
            self.test_button.on_click(on_test_button_clicked)
            
            # Display the button
            display(self.test_button)
            
            self.exp_out.print(f"{self.Name}: Sender node ready - click 'Start Test' to begin measurement")
        else:
            self.exp_out.print(f"{self.Name}: Receiver node ready - waiting for test messages")
        
    def send_test_messages(self):
        """Send 100 test messages with experiment ID, message ID, and timestamp"""
        self.exp_out.print(f"{self.Name}: Starting to send 100 test messages")
        
        # Reset statistics for new test
        self.sent_messages.clear()
        self.received_messages.clear()
        self.total_sent = 0
        self.total_received = 0
        self.duplicate_count = 0
        
        for msg_id in range(100):
            # Get current time with nanosecond precision
            timestamp_ns = time.time_ns()
            
            # Create message with experiment ID, message ID, and timestamp
            message = f"EXP:{self.experiment_id}|MSG:{msg_id}|TIME:{timestamp_ns}"
            
            # Store sent message info for statistics
            self.sent_messages[msg_id] = timestamp_ns
            self.total_sent += 1
            
            # Send message through lower layer
            self.Lower_TR.send(message)
            
        self.exp_out.print(f"{self.Name}: Completed sending {self.total_sent} test messages")
        
    def loop_Rx(self):
        """Receive messages and calculate performance metrics"""
        global thread_running_2L, thread_running_3L
        self.exp_out.print(f"{self.Name}: loop_Rx starting")
        
        # Determine which experiment this belongs to based on experiment_id
        def is_running():
            if self.experiment_id == 1:
                return thread_running_2L
            elif self.experiment_id == 2:
                return thread_running_3L
            else:
                return False
        
        while is_running():
            try:
                # Receive message from lower layer
                msg = self.Lower_TR.receive()
                receive_time_ns = time.time_ns()
                
                # Parse message to extract experiment ID, message ID, and transmission time
                if msg.startswith("EXP:"):
                    parts = msg.split("|")
                    if len(parts) >= 3:
                        exp_id = parts[0].split(":")[1]
                        msg_id = int(parts[1].split(":")[1])
                        send_time_ns = int(parts[2].split(":")[1])
                        
                        # Only process messages from our experiment
                        if exp_id == str(self.experiment_id):
                            self.total_received += 1
                            
                            # Track message for duplicate detection
                            if msg_id not in self.received_messages:
                                self.received_messages[msg_id] = []
                            else:
                                self.duplicate_count += 1
                                self.exp_out.print(f"{self.Name}: Duplicate message detected - MSG:{msg_id}")
                            
                            self.received_messages[msg_id].append(receive_time_ns)
                            
                            # Calculate delay for this message
                            delay_ns = receive_time_ns - send_time_ns
                            delay_ms = delay_ns / 1_000_000  # Convert to milliseconds
                            
                            self.exp_out.print(f"{self.Name} Rx: MSG:{msg_id}, Delay:{delay_ms:.2f}ms")
                            
                            # Check if we should display final metrics
                            # Either we've received all messages, or we've received a reasonable amount and no new messages for a while
                            if self.total_sent > 0 and (len(self.received_messages) >= self.total_sent * 0.8):  # 80% received
                                # Add a small delay and check if this might be the last message
                                if len(self.received_messages) >= 80:  # Reasonable threshold
                                    self.calculate_and_display_metrics()
                                
                else:
                    self.exp_out.print(f"{self.Name} Rx: Non-test message: {msg}")
                    
            except Exception as e:
                self.exp_out.print(f"{self.Name}: Error in loop_Rx: {str(e)}")
                
    def calculate_and_display_metrics(self):
        """Calculate and display performance metrics"""
        if self.total_sent == 0:
            return
            
        # Calculate average delay
        total_delay_ns = 0
        delay_count = 0
        
        for msg_id, timestamps in self.received_messages.items():
            if msg_id in self.sent_messages and timestamps:
                # Use first received timestamp for delay calculation
                send_time = self.sent_messages[msg_id]
                receive_time = timestamps[0]
                delay_ns = receive_time - send_time
                total_delay_ns += delay_ns
                delay_count += 1
        
        avg_delay_ms = (total_delay_ns / delay_count / 1_000_000) if delay_count > 0 else 0
        
        # Calculate message loss rate
        unique_received = len(self.received_messages)
        loss_rate = ((self.total_sent - unique_received) / self.total_sent) * 100
        
        # Calculate message duplication rate
        duplication_rate = (self.duplicate_count / self.total_received) * 100 if self.total_received > 0 else 0
        
        # Display results
        self.exp_out.print(f"\n{self.Name} - PERFORMANCE METRICS:")
        self.exp_out.print(f"Total messages sent: {self.total_sent}")
        self.exp_out.print(f"Unique messages received: {unique_received}")
        self.exp_out.print(f"Total messages received: {self.total_received}")
        self.exp_out.print(f"Average delay: {avg_delay_ms:.2f} ms")
        self.exp_out.print(f"Message loss rate: {loss_rate:.2f}%")
        self.exp_out.print(f"Message duplication rate: {duplication_rate:.2f}%")
        self.exp_out.print(f"Duplicate messages detected: {self.duplicate_count}")
        self.exp_out.print("-" * 50)

In [8]:
thread_running = False
bufferSize = 1024

exp_out = EXP_Output()

AP_local_1  = ("127.0.0.1", 30000)
AP_remote_1 = ("127.0.0.1", 31111)
Socket_1    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_1       = PLE_TR("PLE_Alice", Socket_1, AP_remote_1, AP_local_1, exp_out)

DLE_1 = DLE_TR_FSM("DLE_Alice", PLE_1, exp_out)

ALE_0 = ALE_TextInput(exp_out)
ALE_1 = ALE_TR("ALE_Alice", ALE_0, DLE_1, exp_out)

Text(value='', description='To send:', placeholder='Type something and press enter')

Button(description='Submit', style=ButtonStyle(), tooltip='Click me to submit a string')

19:07:37> ALE_Alice: loop_Tx starting
19:07:37> ALE_Alice: loop_Rx starting
19:07:37> DLE_Alice: loop_Tx starting
19:07:37> DLE_Alice: loop_Rx starting
19:07:37> PLE_Alice: loop_Tx starting
19:07:37> PLE_Alice: loop_Rx starting
19:07:37> ALE_Bob: loop_Tx starting
19:07:37> ALE_Bob: loop_Rx starting
19:07:37> DLE_Bob: loop_Tx starting
19:07:37> DLE_Bob: loop_Rx starting
19:07:37> PLE_Bob: loop_Tx starting
19:07:37> PLE_Bob: loop_Rx starting
19:07:37> DLE_Alice: loop_FSM starting
19:07:37> DLE_Bob: loop_FSM starting
19:07:37> DLE_Alice: loop_Timer starting
19:07:37> DLE_Bob: loop_Timer starting


In [9]:
AP_local_2  = ("127.0.0.1", 31111)
AP_remote_2 = ("127.0.0.1", 30000)
Socket_2    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_2       = PLE_TR("PLE_Bob", Socket_2, AP_remote_2, AP_local_2, exp_out)

DLE_2 = DLE_TR_FSM("DLE_Bob", PLE_2, exp_out)

ALE_3 = ALE_TextInput(exp_out)
ALE_2 = ALE_TR("ALE_Bob", ALE_3, DLE_2, exp_out)

Text(value='', description='To send:', placeholder='Type something and press enter')

Button(description='Submit', style=ButtonStyle(), tooltip='Click me to submit a string')

In [10]:
t1_1 = Thread(target = ALE_1.loop_Tx, args = ()) 
t2_1 = Thread(target = ALE_1.loop_Rx, args = ()) 
t3_1 = Thread(target = DLE_1.loop_Tx, args = ())
t4_1 = Thread(target = DLE_1.loop_Rx, args = ())
t5_1 = Thread(target = PLE_1.loop_Tx, args = ()) 
t6_1 = Thread(target = PLE_1.loop_Rx, args = ())
f1_1 = Thread(target = DLE_1.loop_FSM, args = ())
f2_1 = Thread(target = DLE_1.loop_timer, args = ())

t1_2 = Thread(target = ALE_2.loop_Tx, args = ()) 
t2_2 = Thread(target = ALE_2.loop_Rx, args = ()) 
t3_2 = Thread(target = DLE_2.loop_Tx, args = ())
t4_2 = Thread(target = DLE_2.loop_Rx, args = ())
t5_2 = Thread(target = PLE_2.loop_Tx, args = ()) 
t6_2 = Thread(target = PLE_2.loop_Rx, args = ()) 
f1_2 = Thread(target = DLE_2.loop_FSM, args = ())
f2_2 = Thread(target = DLE_2.loop_timer, args = ())

thread_running = True

t1_1.start()
t2_1.start()
t3_1.start()
t4_1.start()
t5_1.start()
t6_1.start()
t1_2.start()
t2_2.start()
t3_2.start()
t4_2.start()
t5_2.start()
t6_2.start()
f1_1.start()
f1_2.start()
f2_1.start()
f2_2.start()

In [11]:
# Two-layer experiment: MLE_TR -> PLE_TR
exp_out_2layer = EXP_Output()

# Node 1 (Alice) - Two layers - SENDER
AP_local_2L_1  = ("127.0.0.1", 32000)
AP_remote_2L_1 = ("127.0.0.1", 33111)
Socket_2L_1    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_2L_1       = PLE_TR("PLE_Alice_2L", Socket_2L_1, AP_remote_2L_1, AP_local_2L_1, exp_out_2layer)
MLE_2L_1       = MLE_TR("MLE_Alice_2L", PLE_2L_1, exp_out_2layer, experiment_id=1, is_sender=True)

# Node 2 (Bob) - Two layers - RECEIVER
AP_local_2L_2  = ("127.0.0.1", 33111)
AP_remote_2L_2 = ("127.0.0.1", 32000)
Socket_2L_2    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_2L_2       = PLE_TR("PLE_Bob_2L", Socket_2L_2, AP_remote_2L_2, AP_local_2L_2, exp_out_2layer)
MLE_2L_2       = MLE_TR("MLE_Bob_2L", PLE_2L_2, exp_out_2layer, experiment_id=1, is_sender=False)

Button(description='Start Test', style=ButtonStyle())

19:07:37> MLE_Alice_2L: Sender node ready - click 'Start Test' to begin measurement
19:07:37> MLE_Bob_2L: Receiver node ready - waiting for test messages
19:07:37> MLE_Alice_2L: loop_Rx starting
19:07:37> MLE_Bob_2L: loop_Rx starting
19:07:37> PLE_Alice_2L: loop_Tx starting
19:07:37> PLE_Alice_2L: loop_Rx starting
19:07:37> PLE_Bob_2L: loop_Tx starting
19:07:37> PLE_Bob_2L: loop_Rx starting
19:07:37> Two-layer experiment threads started
19:07:46> MLE_Alice_2L: Starting to send 100 test messages
19:07:46> MLE_Alice_2L: Completed sending 100 test messages
19:07:46> PLE_Alice_2L Tx: message: EXP:1|MSG:0|TIME:1760742466461550000
19:07:46> packet loss
19:07:46> PLE_Alice_2L Tx: message: EXP:1|MSG:1|TIME:1760742466461580000
19:07:46> PLE_Alice_2L Tx: message: EXP:1|MSG:2|TIME:1760742466461584000
19:07:46> PLE_Alice_2L Tx: message: EXP:1|MSG:3|TIME:1760742466461587000
19:07:46> PLE_Alice_2L Tx: message: EXP:1|MSG:4|TIME:1760742466461590000
19:07:46> PLE_Alice_2L Tx: message: EXP:1|MSG:5|TIME:

In [12]:
# Three-layer experiment: MLE_TR -> DLE_TR -> PLE_TR
exp_out_3layer = EXP_Output()

# Node 1 (Alice) - Three layers - SENDER
AP_local_3L_1  = ("127.0.0.1", 34000)
AP_remote_3L_1 = ("127.0.0.1", 35111)
Socket_3L_1    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_3L_1       = PLE_TR("PLE_Alice_3L", Socket_3L_1, AP_remote_3L_1, AP_local_3L_1, exp_out_3layer)
DLE_3L_1       = DLE_TR_FSM("DLE_Alice_3L", PLE_3L_1, exp_out_3layer)
MLE_3L_1       = MLE_TR("MLE_Alice_3L", DLE_3L_1, exp_out_3layer, experiment_id=2, is_sender=True)

# Node 2 (Bob) - Three layers - RECEIVER
AP_local_3L_2  = ("127.0.0.1", 35111)
AP_remote_3L_2 = ("127.0.0.1", 34000)
Socket_3L_2    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_3L_2       = PLE_TR("PLE_Bob_3L", Socket_3L_2, AP_remote_3L_2, AP_local_3L_2, exp_out_3layer)
DLE_3L_2       = DLE_TR_FSM("DLE_Bob_3L", PLE_3L_2, exp_out_3layer)
MLE_3L_2       = MLE_TR("MLE_Bob_3L", DLE_3L_2, exp_out_3layer, experiment_id=2, is_sender=False)

Button(description='Start Test', style=ButtonStyle())

19:07:37> MLE_Alice_3L: Sender node ready - click 'Start Test' to begin measurement
19:07:37> MLE_Bob_3L: Receiver node ready - waiting for test messages
19:07:37> MLE_Alice_3L: loop_Rx starting
19:07:37> MLE_Bob_3L: loop_Rx starting
19:07:37> DLE_Alice_3L: loop_Tx starting
19:07:37> DLE_Alice_3L: loop_Rx starting
19:07:37> DLE_Alice_3L: loop_FSM starting
19:07:37> DLE_Alice_3L: loop_Timer starting
19:07:37> DLE_Bob_3L: loop_Tx starting
19:07:37> DLE_Bob_3L: loop_Rx starting
19:07:37> DLE_Bob_3L: loop_FSM starting
19:07:37> DLE_Bob_3L: loop_Timer starting
19:07:37> PLE_Alice_3L: loop_Tx starting
19:07:37> PLE_Alice_3L: loop_Rx starting
19:07:37> PLE_Bob_3L: loop_Tx starting
19:07:37> PLE_Bob_3L: loop_Rx starting
19:07:37> Three-layer experiment threads started


In [13]:
# Thread management for Two-layer experiment
def start_2layer_experiment():
    global thread_running_2L
    thread_running_2L = True
    
    # Start threads for two-layer experiment
    t_mle_rx_1_2L = Thread(target=MLE_2L_1.loop_Rx, args=())
    t_mle_rx_2_2L = Thread(target=MLE_2L_2.loop_Rx, args=())
    t_ple_tx_1_2L = Thread(target=PLE_2L_1.loop_Tx, args=())
    t_ple_rx_1_2L = Thread(target=PLE_2L_1.loop_Rx, args=())
    t_ple_tx_2_2L = Thread(target=PLE_2L_2.loop_Tx, args=())
    t_ple_rx_2_2L = Thread(target=PLE_2L_2.loop_Rx, args=())
    
    t_mle_rx_1_2L.start()
    t_mle_rx_2_2L.start()
    t_ple_tx_1_2L.start()
    t_ple_rx_1_2L.start()
    t_ple_tx_2_2L.start()
    t_ple_rx_2_2L.start()
    
    exp_out_2layer.print("Two-layer experiment threads started")
    
def stop_2layer_experiment():
    global thread_running_2L
    thread_running_2L = False
    exp_out_2layer.print("Two-layer experiment stopped")

# Initialize thread control
thread_running_2L = False

In [14]:
# Thread management for Three-layer experiment
def start_3layer_experiment():
    global thread_running_3L
    thread_running_3L = True
    
    # Start threads for three-layer experiment
    t_mle_rx_1_3L = Thread(target=MLE_3L_1.loop_Rx, args=())
    t_mle_rx_2_3L = Thread(target=MLE_3L_2.loop_Rx, args=())
    t_dle_tx_1_3L = Thread(target=DLE_3L_1.loop_Tx, args=())
    t_dle_rx_1_3L = Thread(target=DLE_3L_1.loop_Rx, args=())
    t_dle_fsm_1_3L = Thread(target=DLE_3L_1.loop_FSM, args=())
    t_dle_timer_1_3L = Thread(target=DLE_3L_1.loop_timer, args=())
    t_dle_tx_2_3L = Thread(target=DLE_3L_2.loop_Tx, args=())
    t_dle_rx_2_3L = Thread(target=DLE_3L_2.loop_Rx, args=())
    t_dle_fsm_2_3L = Thread(target=DLE_3L_2.loop_FSM, args=())
    t_dle_timer_2_3L = Thread(target=DLE_3L_2.loop_timer, args=())
    t_ple_tx_1_3L = Thread(target=PLE_3L_1.loop_Tx, args=())
    t_ple_rx_1_3L = Thread(target=PLE_3L_1.loop_Rx, args=())
    t_ple_tx_2_3L = Thread(target=PLE_3L_2.loop_Tx, args=())
    t_ple_rx_2_3L = Thread(target=PLE_3L_2.loop_Rx, args=())
    
    t_mle_rx_1_3L.start()
    t_mle_rx_2_3L.start()
    t_dle_tx_1_3L.start()
    t_dle_rx_1_3L.start()
    t_dle_fsm_1_3L.start()
    t_dle_timer_1_3L.start()
    t_dle_tx_2_3L.start()
    t_dle_rx_2_3L.start()
    t_dle_fsm_2_3L.start()
    t_dle_timer_2_3L.start()
    t_ple_tx_1_3L.start()
    t_ple_rx_1_3L.start()
    t_ple_tx_2_3L.start()
    t_ple_rx_2_3L.start()
    
    exp_out_3layer.print("Three-layer experiment threads started")
    
def stop_3layer_experiment():
    global thread_running_3L
    thread_running_3L = False
    exp_out_3layer.print("Three-layer experiment stopped")

# Initialize thread control
thread_running_3L = False

In [ ]:
# STEP 1: Run this cell to start the two-layer measurement experiment
print("Starting two-layer measurement experiment...")
start_2layer_experiment()

Starting two-layer measurement experiment...
✓ Two-layer experiment is now running!
✓ Alice (sender) -> Bob (receiver)
✓ Press the 'Start Test' button on MLE_Alice_2L to begin the measurement
✓ Bob will receive and measure the messages automatically


In [ ]:
# STEP 2: Run this cell to start the three-layer measurement experiment
print("Starting three-layer measurement experiment...")
start_3layer_experiment()

Starting three-layer measurement experiment...
✓ Three-layer experiment is now running!
✓ Alice (sender) -> Bob (receiver)
✓ Press the 'Start Test' button on MLE_Alice_3L to begin the measurement
✓ Bob will receive and measure the messages automatically
